In [1]:
import mysql.connector
import pandas as pd
import mysql.connector.errorcode

In [3]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()

In [6]:
#Create table order logs
order_logs = """
CREATE TABLE order_logs (
               id INT AUTO_INCREMENT PRIMARY KEY, 
               log_message TEXT, 
               created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
try:
    cursor.execute(order_logs)
    print("Table order_logs created successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")

#Commit changes
connection.commit()

In [7]:
trigger_log_delete_order = """
CREATE TRIGGER log_delete_order
AFTER DELETE ON orders
FOR EACH ROW
BEGIN
INSERT INTO order_logs (log_message)
VALUES (CONCAT('Old order ID: ', OLD.id, ' - total: ', OLD.total));
END;
"""
try:
    cursor.execute(trigger_log_delete_order)
    print("Trigger log_delete_order inserted successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")

# Đóng kết nối
cursor.close()
connection.close()

Trigger log_delete_order inserted successfully


In [15]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()

try:
    cursor.execute('DELETE FROM order_items oi WHERE oi.order_id in (Select id from orders where id = 1)')
    cursor.execute('Delete from orders where id = 1')
    connection.commit()
except mysql.connector.Error as err:
    print(f"Error: {err}")

In [23]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()
df = pd.read_sql_query("Select * from order_logs", connection)
df.head()



C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_3332\2502494356.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("Select * from order_logs", connection)


,id,log_message,created_at
0,1,Old order ID: 1 - total: 500.00,2025-04-10 16:19:51


In [24]:
connection.close()
cursor.close()

True

In [26]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()

trigger_update_order_total = """
CREATE TRIGGER update_order_total
AFTER UPDATE ON order_items
FOR EACH ROW
BEGIN
UPDATE orders SET total = (SELECT SUM(price * quantity) FROM order_items WHERE order_id = NEW.order_id)
WHERE id = NEW.order_id;
END;
"""

try:
    cursor.execute(trigger_update_order_total)
except mysql.connector.Error as err:
    print(f"Error: {err}")
connection.close()
cursor.close()

Error: 1359 (HY000): Trigger already exists


True

In [34]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()

trigger_price_not_exceed_1000 = """
CREATE TRIGGER price_not_exceed_1000
BEFORE UPDATE ON products
FOR EACH ROW
BEGIN
    IF NEW.price > 1000 THEN
        SIGNAL SQLSTATE '45000' 
        SET MESSAGE_TEXT = 'Price cannot exceed 1000$';
    END IF;
END;
"""

try:
    cursor.execute(trigger_price_not_exceed_1000)
except mysql.connector.Error as err:
    print(f"Error: {err}")


connection.commit()

Error: 1360 (HY000): Trigger does not exist


In [35]:
query = """
UPDATE products SET price = 1003 WHERE id = 2;
"""

try:
    cursor.execute(query)
except mysql.connector.Error as err:
    print(f"Error: {err}")

Error: 1644 (45000): Price cannot exceed 1000$


In [36]:
query = """
Create table customers_deleted (
    id INT,
    name VARCHAR(255),
    city VARCHAR(255),
    deleted_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

try:
    cursor.execute(query)
    connection.commit()
except mysql.connector.Error as err:
    print(f"Error: {err}")

In [37]:
trigger_delete_customer = """
Create trigger delete_customer
After delete on customers
for each row
begin
    insert into customers_deleted (id, name, city, deleted_at)
    values ('Old id: ', old.id, ' - Old name: ', old.name, ' - Old city: ', old.city, ' - Deleted at: ', now());
end;
"""
try:
    cursor.execute(trigger_delete_customer)
    connection.commit()
except mysql.connector.Error as err:
    print(f"Error: {err}")

In [47]:

try:
    cursor.execute('DELETE FROM order_items WHERE customer_id IN (SELECT id FROM orders WHERE customer_id = 1);')
    cursor.execute('DELETE FROM orders WHERE customer_id = 1;')
    cursor.execute('DELETE FROM customers WHERE id = 1;')
    connection.commit()
except mysql.connector.Error as err:
    print(f"Error: {err}")


Error: 1054 (42S22): Unknown column 'customer_id' in 'IN/ALL/ANY subquery'


In [44]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()

query = """select * from customers_deleted;"""
df = pd.read_sql_query(query, connection)
print(df)





Empty DataFrame
Columns: [id, name, city, deleted_at]
Index: []


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_3332\2729908622.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [2]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()


In [3]:
query = """
Create table order_alert (
    id INT PRIMARY KEY,
    alert_message TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
try:
    cursor.execute(query)
    connection.commit()
except mysql.connector.Error as err:
    print(f"Error: {err}")

In [5]:
trigger_order_alert = """
Create trigger order_alert
After insert on orders
for each row
begin
    if new.total > 1000 and (Select city from customers where id = new.customer_id) = 'Hanoi' then
        insert into order_alert (alert_message) 
        values ('Order total is greater than 1000');
    end if;
end;
"""
try:
    cursor.execute(trigger_order_alert)
    connection.commit()
except mysql.connector.Error as err:
    print(f"Error: {err}")